###Initializing a SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NLP').getOrCreate()

###Loading the dataset

In [4]:
df = spark.read.csv('/FileStore/tables/SMSSpamCollection', inferSchema=True, sep='\t')
df = df.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
df.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

###Feature engineering

In [6]:
from pyspark.sql.functions import length
df = df.withColumn('length', length(df['text']))
df.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [7]:
df.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

doc 1 'i, like, apples, i'

tf-idf

i = 2 -> 2 * log(2/2) -> 0

like = 1 -> 1 * log(2/1) -> 0.3 ('like' is a relatively relevent string in this doc)

apples = 1 -> 1 * log(2/2) -> 0

hate = 0 -> 0 * log(2/1) -> 0

doc 2 'i, hate, apples'

i = 1 -> 1 * log(2/2) -> 0

like = 0 -> 0 * log(2/1) -> 0

apples = 1 -> 1 * log(2/2) -> 0

hate = 1 -> 1 * log(2/1) -> 0.3 ('hate' is a relatively relevent string in this doc)

idf

i = 2

like = 1

apples = 2

hate = 1

N = 2

###Formatting the text columng

In [10]:
from pyspark.ml.feature import StringIndexer, Tokenizer, StopWordsRemover, CountVectorizer, IDF, VectorAssembler
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')
# df_1 = ham_spam_to_numeric.fit(df).transform(df)
# df_1.show()
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
# df_2 = tokenizer.transform(df_1)
# df_2.show()
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='token_stop')
# df_3 = stop_remove.transform(df_2)
# df_3.show()
count_vec = CountVectorizer(inputCol='token_stop', outputCol='count_vec')
# df_4 = count_vec.fit(df_3).transform(df_3)
# df_4.show()
idf = IDF(inputCol='count_vec', outputCol='tf-idf')
# df_5 = idf.fit(df_4).transform(df_4)
# df_5.show()
transformed_df = VectorAssembler(inputCols=['length', 'tf-idf'], outputCol='features')
# df_6 = transformed_df.transform(df_5)
# df_6.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
class| text|length|label| token_text| token_stop| count_vec| tf-idf| features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 ham|Go until jurong p...| 111| 0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[0,8,12,32...|
 ham|Ok lar... Joking ...| 29| 0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,1,25,29...|
 spam|Free entry in 2 a...| 155| 1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[0,3,14,20...|
 ham|U dun say so earl...| 49| 0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,80,1...|(13424,[0,1,71,81...|
 ham|Nah I don't think...| 61| 0.0|[nah, i, don't, t...|[nah, think, goes...|(13423,[36,134,31...|(13423,[36,134,31...|(13424,[0,37,135,...|
 spam|FreeMsg Hey there...| 147| 1.0|[freemsg, hey, th...|[freemsg, hey, da...|(13423,[10,60,139...|(13423,[10,60,139...|(13424,[0,11,61,1...|
 ham|Even my brother i...| 77| 0.0|[even, my, brothe...|[even, brother, l...|(13423,[10,53,103...|(13423,[10,53,103...|(13424,[0,11,54,1...|
 ham|As per your reque...| 160| 0.0|[as, per, your, r...|[per, request, 'm...|(13423,[125,184,4...|(13423,[125,184,4...|(13424,[0,126,185...|
 spam|WINNER!! As a val...| 157| 1.0|[winner!!, as, a,...|[winner!!, valued...|(13423,[1,47,118,...|(13423,[1,47,118,...|(13424,[0,2,48,11...|
 spam|Had your mobile 1...| 154| 1.0|[had, your, mobil...|[mobile, 11, mont...|(13423,[0,1,13,27...|(13423,[0,1,13,27...|(13424,[0,1,2,14,...|
 ham|I'm gonna be home...| 109| 0.0|[i'm, gonna, be, ...|[gonna, home, soo...|(13423,[18,43,120...|(13423,[18,43,120...|(13424,[0,19,44,1...|
 spam|SIX chances to wi...| 136| 1.0|[six, chances, to...|[six, chances, wi...|(13423,[8,17,37,8...|(13423,[8,17,37,8...|(13424,[0,9,18,38...|
 spam|URGENT! You have ...| 155| 1.0|[urgent!, you, ha...|[urgent!, won, 1,...|(13423,[13,30,47,...|(13423,[13,30,47,...|(13424,[0,14,31,4...|
 ham|I've been searchi...| 196| 0.0|[i've, been, sear...|[searching, right...|(13423,[39,96,217...|(13423,[39,96,217...|(13424,[0,40,97,2...|
 ham|I HAVE A DATE ON ...| 35| 0.0|[i, have, a, date...|[date, sunday, wi...|(13423,[552,1697,...|(13423,[552,1697,...|(13424,[0,553,169...|
 spam|XXXMobileMovieClu...| 149| 1.0|[xxxmobilemoviecl...|[xxxmobilemoviecl...|(13423,[30,109,11...|(13423,[30,109,11...|(13424,[0,31,110,...|
 ham|Oh k...i'm watchi...| 26| 0.0|[oh, k...i'm, wat...|[oh, k...i'm, wat...|(13423,[82,214,47...|(13423,[82,214,47...|(13424,[0,83,215,...|
 ham|Eh u remember how...| 81| 0.0|[eh, u, remember,...|[eh, u, remember,...|(13423,[0,2,49,13...|(13423,[0,2,49,13...|(13424,[0,1,3,50,...|
 ham|Fine if thats th...| 56| 0.0|[fine, if, thats...|[fine, thats, wa...|(13423,[0,74,105,...|(13423,[0,74,105,...|(13424,[0,1,75,10...|
 spam|England v Macedon...| 155| 1.0|[england, v, mace...|[england, v, mace...|(13423,[4,30,33,5...|(13423,[4,30,33,5...|(13424,[0,5,31,34...|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [11]:
from pyspark.ml import Pipeline
df_pipe = Pipeline(stages=[ham_spam_to_numeric,
                           tokenizer,
                           stop_remove,
                           count_vec,
                           idf,
                           transformed_df])
final_df = df_pipe.fit(df).transform(df).select('label', 'features')
final_df.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[0,8,12,32...|
 0.0|(13424,[0,1,25,29...|
 1.0|(13424,[0,3,14,20...|
 0.0|(13424,[0,1,71,81...|
 0.0|(13424,[0,37,135,...|
 1.0|(13424,[0,11,61,1...|
 0.0|(13424,[0,11,54,1...|
 0.0|(13424,[0,126,185...|
 1.0|(13424,[0,2,48,11...|
 1.0|(13424,[0,1,2,14,...|
 0.0|(13424,[0,19,44,1...|
 1.0|(13424,[0,9,18,38...|
 1.0|(13424,[0,14,31,4...|
 0.0|(13424,[0,40,97,2...|
 0.0|(13424,[0,553,169...|
 1.0|(13424,[0,31,110,...|
 0.0|(13424,[0,83,215,...|
 0.0|(13424,[0,1,3,50,...|
 0.0|(13424,[0,1,75,10...|
 1.0|(13424,[0,5,31,34...|
+-----+--------------------+
only showing top 20 rows

###Splitting the dataset

In [13]:
train_data, test_data = final_df.randomSplit([0.7, 0.3])

###Building the ML model

In [15]:
from pyspark.ml.classification import NaiveBayes
classifier = NaiveBayes(featuresCol='features', labelCol='label', predictionCol='prediction')
fittied_classifer = classifier.fit(train_data)

###Predicting the test data

In [17]:
preds = fittied_classifer.transform(test_data)
preds.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,3,8...|[-811.58126358978...|[1.0,1.1870929167...| 0.0|
 0.0|(13424,[0,1,2,6,1...|[-1004.2536555575...|[1.0,9.9687763567...| 0.0|
 0.0|(13424,[0,1,2,8,9...|[-886.65484041376...|[1.0,1.4573713517...| 0.0|
 0.0|(13424,[0,1,2,8,9...|[-1169.9441674290...|[1.0,4.6484715559...| 0.0|
 0.0|(13424,[0,1,2,8,1...|[-665.15817956926...|[1.0,6.6046593626...| 0.0|
 0.0|(13424,[0,1,2,10,...|[-535.95986127579...|[1.0,2.9507903324...| 0.0|
 0.0|(13424,[0,1,2,12,...|[-904.70222111608...|[1.0,3.1153084793...| 0.0|
 0.0|(13424,[0,1,2,15,...|[-1380.1961334642...|[1.0,1.8029251227...| 0.0|
 0.0|(13424,[0,1,2,15,...|[-217.25100994637...|[1.0,1.1390791090...| 0.0|
 0.0|(13424,[0,1,2,21,...|[-984.99230055877...|[0.99999978426289...| 0.0|
 0.0|(13424,[0,1,2,24,...|[-1289.4363746907...|[1.0,2.0169285296...| 0.0|
 0.0|(13424,[0,1,2,25,...|[-340.59059890977...|[1.0,2.5697820648...| 0.0|
 0.0|(13424,[0,1,2,32,...|[-345.21796861660...|[1.0,3.6902631427...| 0.0|
 0.0|(13424,[0,1,2,47,...|[-1136.3575282709...|[1.20026835332069...| 1.0|
 0.0|(13424,[0,1,2,147...|[-254.99821981792...|[0.07085331104352...| 1.0|
 0.0|(13424,[0,1,3,4,6...|[-520.27662591988...|[0.99999999988839...| 0.0|
 0.0|(13424,[0,1,3,4,7...|[-3297.9989953384...|[1.0,3.7847268522...| 0.0|
 0.0|(13424,[0,1,3,5,6...|[-2494.1778078113...|[1.0,1.2129570174...| 0.0|
 0.0|(13424,[0,1,3,5,9...|[-1306.3515943144...|[1.0,3.8393130263...| 0.0|
 0.0|(13424,[0,1,3,5,4...|[-1929.9203135062...|[1.0,8.0264858232...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

###Evaluating the model

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
area_under_curve = evaluator.evaluate(preds)
accuracy = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='label')
accuracy = accuracy.evaluate(preds)
print(area_under_curve)
print(accuracy)

0.9330873434804146
0.9059316956261234